In [2]:
from netCDF4 import Dataset as NetCDFFile
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from urllib.request import urlretrieve
import os
from torch.utils.data import Dataset, DataLoader
from skimage.transform import resize
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils as vutils
from torch.autograd import Variable
from torch import LongTensor, FloatTensor

## 下载数据,更改部分代码实现测试集与训练集分别生成

In [21]:
datetime(2017,1,1)-datetime(2010,1,1)
# datetime(2020,1,1)-datetime(2017,1,1)

datetime.timedelta(days=2557)

In [24]:
    
class NWSDataset(Dataset):
    """
    NWS Dataset
    """

    def __init__(
        self, path='../train', prefix="sst"  # train or test
    ):
        self.path = path
        self.files = [
                        f
                        for f in os.listdir(path)
                        if f.startswith(prefix) and os.path.isfile(os.path.join(path, f))
                    ]
        self.maxclip = 15  # sst range:  ~[-2,13.5]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, item):
        nc = NetCDFFile(
            os.path.join(self.path,self.files[item])
        )
        sst = nc.variables["sst"]
        data = sst[:] + 5  ## min(i.e. nan) = -5
        data = FloatTensor(resize(data, (64, 64)))  ## raw size -> 64*64
        data = data.view((-1, 64, 64))
        data = torch.clamp(data, max=self.maxclip)  ## set upper boundary is 100
        data = data / self.maxclip
        data = (data * 2) - 1  # value range: [-1, 1]
        return data



In [25]:
## batch_size：选择多少数据打成一个包  batch =  NWSDataset(len) / batch-size
dataloader = DataLoader(NWSDataset(), batch_size=256, shuffle=True)
data = []
for i in dataloader:
    data.append(i)
data = torch.cat(data, 0)
sums = data.sum(dim = (1, 2, 3)).detach().cpu().numpy().argsort()[::-1].copy()
torch.save(data.data[sums], '../data/train.pt')  # train or test
print(data.data[sums].shape)
print(sums.shape)
# train n:2557
# test n:1462

torch.Size([2557, 1, 64, 64])
(2557,)


In [18]:
data2=torch.load('../data/train.pt')
n=data2.shape[0]
data2=data2[:,:,:,:].detach().numpy()
data2=data2.squeeze()
### 逆归一化
data2=(data2+1)/2
data2=data2*15
data2 = FloatTensor(resize(data2, (n,601, 205)))
# idx = data2>(0)
print(data2.shape)
print(data2[1,:,:])

torch.Size([2557, 601, 205])
tensor([[ 4.0222,  3.7740,  3.9718,  ...,  4.9583,  4.9628,  4.9572],
        [ 3.7986,  3.5147,  3.7409,  ...,  4.9461,  4.9485,  4.9455],
        [ 3.5750,  3.2554,  3.5101,  ...,  4.9339,  4.9342,  4.9338],
        ...,
        [11.8192, 13.1904, 12.0977,  ..., 10.3824, 10.3988, 10.3782],
        [11.9643, 13.3249, 12.2407,  ..., 10.3733, 10.3877, 10.3696],
        [12.1094, 13.4594, 12.3836,  ..., 10.3641, 10.3767, 10.3609]])
